In [1]:
# This requires you to have pins 567 up to flash
%esptool erase

Executing:
  esptool.py --port /dev/ttyUSB0 erase_flash

esptool.py v2.8
Serial port /dev/ttyUSB0
Connecting....
[Press the PRG button now if required]
Detecting chip type... ESP8266
Chip is ESP8266EX
Features: WiFi
Crystal is 26MHz
MAC: dc:4f:22:60:bd:cd
Uploading stub...
Running stub...
Stub running...
Erasing flash (this may take a while)...
Chip erase completed successfully in 16.8s
Hard resetting via RTS pin...


In [2]:
# This requires you to have pins 567 up to flash
# This firmware was built with mqtt_as from https://github.com/goatchurchprime/jupyter_micropython_developer_notebooks/blob/master/mqtt_async_projects/esp8266_commissioning_mqtt_as.ipynb
%esptool esp8266 bin/firmware-combined.bin

Executing:
  esptool.py --port /dev/ttyUSB0 --baud 460800 write_flash --flash_size=detect -fm dio 0 bin/firmware-combined.bin

esptool.py v2.8
Serial port /dev/ttyUSB0
Connecting....
[Press the PRG button now if required]
Detecting chip type... ESP8266
Chip is ESP8266EX
Features: WiFi
Crystal is 26MHz
MAC: dc:4f:22:60:bd:cd
Uploading stub...
Running stub...
Stub running...
Changing baud rate to 460800
Changed.
Configuring flash size...
Auto-detected Flash size: 4MB
Flash params set to 0x0240
Compressed 624400 bytes to 406630...
Wrote 624400 bytes (406630 compressed) at 0x00000000 in 9.7 seconds (effective 512.4 kbit/s)...
Hash of data verified.

Leaving...
Hard resetting via RTS pin...


In [4]:
# now just the 56 switches
%serialconnect



***Connection broken [Input/output error]
You may need to reconnect
Closing serial Serial<id=0x7fd3e8046d50, open=True>(port='/dev/ttyUSB0', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=0.5, xonxoff=False, rtscts=False, dsrdtr=False)
Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.


In [5]:
help("modules")

__main__          lighthttpserve_funcs                uasyncio/queues   urandom
_boot             lwip              uasyncio/synchro  ure
_onewire          machine           ubinascii         uselect
_webrepl          math              ucollections      usocket
apa102            micropython       ucryptolib        ussl
btree             mqtt_as           uctypes           ustruct
builtins          mqtt_as_timeout   uerrno            utime
collections/deque neopixel          uhashlib          utimeq
dht               network           uheapq            uwebsocket
ds18x20           ntptime           uio               uzlib
esp               onewire           ujson             webrepl
flashbdev         port_diag         umqtt/robust      webrepl_setup
framebuf          sys               umqtt/simple      websocket_helper
gc                uarray            uos
inisetup          uasyncio/__init__ upip
itertools         uasyncio/core     upip_utarfile
Plus any modules on the filesystem


In [6]:
# this is the code for changing the wifi credentials
%sendtofile --source utils.py

Sent 32 lines (969 bytes) to utils.py.


In [7]:
%sendtofile config.txt

wifiname     DoESLiverpool
wifipassword decafbad00
mqttbroker   sensorcity.io
pinled       14
boardname    abdushop2

Sent 5 lines (116 bytes) to config.txt.


In [12]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB1 --baud=115200 
state: 2 -> 3 (0)
Ready.


In [ ]:
#%sendtofile main.py

import time, network, os, sys
from machine import Pin, UART
from mqtt_as import config, MQTTClient
import uasyncio as asyncio

fconfig = dict(x.split()  for x in open("config.txt"))
pinled = Pin(int(fconfig["pinled"]), Pin.OUT)
for i in range(9):
    pinled.value(i%2)
    time.sleep_ms(300)
    
boardname = fconfig["boardname"]
topicserial = boardname+'/serial'
topicip = boardname+'/ip'
topicstatus = boardname+'/status'
topicerror = boardname+'/error'

#os.dupterm(None, 1)
#uart = UART(0, baudrate=115200, rxbuf=240)
#uartdebug = UART(1, baudrate=115200)
uart = None

n = 0            
async def serialtask():
    global n
    buf = memoryview(bytearray(240))
    e = 0
    while True:
        n += 1
        s, e = e, e + (uart and uart.readinto(buf[e:]) or 0)
        while s < e:
            s += 1
            if buf[s-1] == ord('\n'):
                if client.isconnected():
                    await client.publish(topicserial, buf[:s-1])
                    print("message sent")
                buf[:e-s] = buf[s:e]
                s, e = 0, e-s
        if e == len(buf):
            if client.isconnected():
                await client.publish(topicerror, b"BUFFEROVERFLOW")
            e = 0
        await asyncio.sleep_ms(5)

async def onconnecttask(client):
    ipnumber = client._sta_if.ifconfig()[0]
    print("ipnumber", ipnumber)
    await client.publish(topicstatus, "connected", retain=True)
    print("connected")
        
async def connecttask():
    print("connecttask")
    try:
        await client.connect()
    except OSError as e:
        print("client connect error", [e])
        machine.reset()
        return
    
    ipnumber = client._sta_if.ifconfig()[0]
    print("connecttask connected")
    for i in range(19):
        pinled.value(i%2)
        await asyncio.sleep_ms(100)
    n = 0
    while True:
        print("connecttask loop", n, client.isconnected())
        n += 1
        if client.isconnected():
            await client.publish(topicstatus, "on %d"%n, retain=True)
        await asyncio.sleep_ms(3000)

config['server'] = fconfig["mqttbroker"]
config['ssid'] = fconfig["wifiname"]
config['wifi_pw'] = fconfig["wifipassword"]
config['mqttchannel'] = boardname
config['connect_coro'] = onconnecttask
config['will'] = [ topicstatus, "dropped", True]
  
client = MQTTClient(config)
aloop = asyncio.get_event_loop()
aloop.create_task(connecttask())
aloop.create_task(serialtask())
aloop.run_forever()



[leftinbuffer] ['state: 3 -> 5 (10)']
[leftinbuffer] ['add 0']
[leftinbuffer] ['aid 1']
[leftinbuffer] ['cnt ']
[leftinbuffer] ['connected with DoESLiverpool, channel 6']
[leftinbuffer] ['dhcp client start...']
sleep disable
connecttask
scandone
state: 5 -> 2 (b0)
state: 5 -> 0 (2)
reconnect
scandone
state: 0 -> 2 (b0)
state: 2 -> 3 (0)
state: 3 -> 5 (10)
add 0
aid 1
cnt 

connected with DoESLiverpool, channel 6
dhcp client start...
scandone
state: 5 -> 2 (b0)
state: 5 -> 0 (2)
reconnect
scandone
state: 0 -> 2 (b0)
state: 2 -> 3 (0)
state: 3 -> 5 (10)
add 0
aid 1
cnt 

connected with DoESLiverpool, channel 6
dhcp client start...
client connect error [OSError()]
